In [1]:
import pandas as pd
import numpy as np
import sys 
import datetime
import os
import matplotlib as plt
import seaborn as sns

%matplotlib inline

# Process the Weekday data

## Averages data for all of 2019

* Average weekday non-pooled trip totals 
* Average weekday pooled trip totals
* Average weekday fares
* Average weekday travel time


In [5]:
tods = [1,2,3,4,5]
years = [2018,2019,2020]

In [6]:
agg = { 'Trip Seconds':'mean',
       'Trip Miles':'mean','Fare':'mean',
       'Tip':'mean', 'Additional Charges':'mean', 'Trip Total':'mean', 'PRIVATE_TRIPS':'sum','SHARED_TRIPS':'sum',
       'Trips Pooled':'sum'}


In [8]:
df_all = pd.DataFrame()
df_raw = pd.DataFrame()

for year in years:
    print('Working on year ' + str(year))
    for tod in tods:
        print('Working on tod ' + str(tod))
        df = pd.read_hdf('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Old Files/Chicago_TNC_Trips_20_Incomplete.H5', where = 'YEAR == ' + str(year), key = 'Weekday_' + str(tod))

        #df = df[df['MONTH'].isin([1,2])]
        #df = df[df['YEAR']==2019]

        print(df['Trip Seconds'].max())
        print(df['Trip Miles'].max())

        df = df.dropna(subset = ['Pickup Census Tract', 'Dropoff Census Tract'])
        df['DAY'] = df['Trip Start Timestamp'].dt.day
        df['PRIVATE_TRIPS'] = np.where(df['Shared Trip Authorized'] == False, 1, 0)
        df['SHARED_TRIPS'] = np.where(df['Shared Trip Authorized'] == True, 1, 0)  

        ## the filters are higher than the longest trip because the trip could be pooled
        # filter out the trips that are longer than 50 miles becuase the longest possible trip is 35 miles going from O'Hare airport to south east corner
        df2 = df[df['Trip Miles'] <= 50]

        # filter out the trips that are longer than 2 hours becuase the longest possible trip is 1 hours going from O'Hare airport to south east corner
        df2 = df2[df2['Trip Seconds'] <= 7200]

        print(df2['Trip Seconds'].max())
        print(df2['Trip Miles'].max())

        #census tract XXX replaces the trips assigned to census tract XXX because they are likely misasigned. Census tract XXX contains
        #contains O'Hare airport and census tract is adjacent. The trips assigned to census tract XXX are likely trips from the airport.
        df2.loc[df2['Pickup Census Tract'] == 17031770700, 'Pickup Census Tract'] = 17031980000
        df2.loc[df2['Dropoff Census Tract'] == 17031770700, 'Dropoff Census Tract'] = 17031980000


        #trips to census tract XXX are likely misasigned but it is not clear which census tract they should be assigned to.
        #there is only one trip, so it is droped
        df2 = df2[df2['Pickup Census Tract'] != 17031810502]
        df2 = df2[df2['Dropoff Census Tract'] != 17031810502]

       # if df2['MONTH'] == [1,2,3]:
        #    df2['QUARTER'] = 1

        #elif df2['MONTH'] == [4,5,6]:
         #   df2['QUARTER'] = 2

        #elif df2['MONTH'] == [7,8,9]:
         #   df2['QUARTER'] = 3

        #elif df2['MONTTH'] == [10,11,12]:
         #   df2['QUARTER'] = 4

        df2 = df2.groupby(by= ['Pickup Census Tract','Dropoff Census Tract','YEAR', 'MONTH','DAY'], as_index =False).agg(agg)
        df2 = df2.groupby(by= ['Pickup Census Tract','Dropoff Census Tract','YEAR','MONTH'], as_index =False).mean()

        df['TOD'] = tod
        df2['TOD'] = tod

        df_raw = df_raw.append(df)
        df_all = df_all.append(df2)

Working on year 2018
Working on tod 1
39001.0
389.9
7102.0
50.0
Working on tod 2
23417.0
298.6
7081.0
47.7
Working on tod 3
27482.0
295.7
6882.0
49.6
Working on tod 4
32241.0
296.7
6892.0
49.7
Working on tod 5
71040.0
309.9
7006.0
50.0
Working on year 2019
Working on tod 1
66720.0
546.1
7158.0
50.0
Working on tod 2
29520.0
269.7
7140.0
49.0
Working on tod 3
78780.0
338.9
7198.0
50.0
Working on tod 4
47100.0
335.5
7003.0
49.8
Working on tod 5
80520.0
380.0
7192.0
50.0
Working on year 2020
Working on tod 1
18609.0
313.8
5335.0
49.1
Working on tod 2
12488.0
247.5
5543.0
45.3
Working on tod 3
15749.0
266.6
6880.0
50.0
Working on tod 4
13927.0
225.9
7066.0
48.3
Working on tod 5
32694.0
296.6
5973.0
49.5


## Add in the Suppressed Trips

In [7]:
sup_trips = pd.read_csv('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/2019 Suppressed Trips.csv')

In [8]:
df_all.head()

,Pickup Census Tract,Dropoff Census Tract,DAY,Trip Seconds,Trip Miles,Fare,Tip,Additional Charges,Trip Total,PRIVATE_TRIPS,SHARED_TRIPS,Trips Pooled,TOD
0,1.703101e+10,1.703101e+10,16.173913,448.061025,1.381268,4.991460,0.385404,2.373641,7.750505,2.304348,0.652174,2.956522,1
1,1.703101e+10,1.703101e+10,16.300000,282.996230,1.016183,4.055952,0.123175,2.324792,6.503919,2.300000,1.033333,3.666667,1
2,1.703101e+10,1.703101e+10,17.470588,299.877451,1.095098,4.301471,0.284314,1.947059,6.532843,0.764706,0.823529,1.764706,1
3,1.703101e+10,1.703101e+10,13.863636,218.310606,0.703788,3.579545,0.590909,2.125000,6.295455,1.136364,0.500000,1.954545,1
4,1.703101e+10,1.703101e+10,15.692308,326.284799,1.366850,4.922161,0.338828,2.130495,7.391484,1.153846,0.500000,1.923077,1


In [9]:
sup_trips.head()

,Unnamed: 0,GEOID_PICKUP,GEOID_DROPOFF,TOD,SCALED_SUP_PRIVATE_TRIPS,SCALED_SUP_SHARED_TRIPS,Pickup Community Area,Dropoff Community Area
0,0,17031010100,17031010100,1,0.017911,0.007787,1.0,1.0
1,1,17031010100,17031010100,2,0.014015,0.006021,1.0,1.0
2,2,17031010100,17031010100,3,0.553358,0.310965,1.0,1.0
3,3,17031010100,17031010100,4,0.004297,0.002685,1.0,1.0
4,4,17031010100,17031010100,5,0.041810,0.025138,1.0,1.0


In [10]:
df_final = df_all.merge(sup_trips[['GEOID_PICKUP', 'GEOID_DROPOFF', 'SCALED_SUP_PRIVATE_TRIPS', 'SCALED_SUP_SHARED_TRIPS', 'TOD']], how = 'left', left_on = ['Pickup Census Tract','Dropoff Census Tract', 'TOD'],right_on = ['GEOID_PICKUP', 'GEOID_DROPOFF', 'TOD'])

In [11]:
df_final.head()

,Pickup Census Tract,Dropoff Census Tract,DAY,Trip Seconds,Trip Miles,Fare,Tip,Additional Charges,Trip Total,PRIVATE_TRIPS,SHARED_TRIPS,Trips Pooled,TOD,GEOID_PICKUP,GEOID_DROPOFF,SCALED_SUP_PRIVATE_TRIPS,SCALED_SUP_SHARED_TRIPS
0,1.703101e+10,1.703101e+10,16.173913,448.061025,1.381268,4.991460,0.385404,2.373641,7.750505,2.304348,0.652174,2.956522,1,1.703101e+10,1.703101e+10,0.017911,0.007787
1,1.703101e+10,1.703101e+10,16.300000,282.996230,1.016183,4.055952,0.123175,2.324792,6.503919,2.300000,1.033333,3.666667,1,1.703101e+10,1.703101e+10,0.607989,0.264343
2,1.703101e+10,1.703101e+10,17.470588,299.877451,1.095098,4.301471,0.284314,1.947059,6.532843,0.764706,0.823529,1.764706,1,1.703101e+10,1.703101e+10,0.084647,0.036803
3,1.703101e+10,1.703101e+10,13.863636,218.310606,0.703788,3.579545,0.590909,2.125000,6.295455,1.136364,0.500000,1.954545,1,1.703101e+10,1.703101e+10,0.031673,0.013771
4,1.703101e+10,1.703101e+10,15.692308,326.284799,1.366850,4.922161,0.338828,2.130495,7.391484,1.153846,0.500000,1.923077,1,1.703101e+10,1.703101e+10,0.103718,0.045095


In [12]:
df_final['SHARED_TRIPS'] = df_final['SHARED_TRIPS'] + df_final['SCALED_SUP_SHARED_TRIPS']
df_final['PRIVATE_TRIPS'] = df_final['PRIVATE_TRIPS'] + df_final['SCALED_SUP_PRIVATE_TRIPS']
df_final['ALL_TRIPS'] = df_final['PRIVATE_TRIPS'] + df_final['SHARED_TRIPS']

# Merge RH Data to Empty Chicago OD Matrix

In [13]:
empty = pd.read_csv('Inputs/Chicago Ride-Hailing/Empty_Chicago_Matrix.csv')

In [14]:
df_final.head()

,Pickup Census Tract,Dropoff Census Tract,DAY,Trip Seconds,Trip Miles,Fare,Tip,Additional Charges,Trip Total,PRIVATE_TRIPS,SHARED_TRIPS,Trips Pooled,TOD,GEOID_PICKUP,GEOID_DROPOFF,SCALED_SUP_PRIVATE_TRIPS,SCALED_SUP_SHARED_TRIPS,ALL_TRIPS
0,1.703101e+10,1.703101e+10,16.173913,448.061025,1.381268,4.991460,0.385404,2.373641,7.750505,2.322259,0.659961,2.956522,1,1.703101e+10,1.703101e+10,0.017911,0.007787,2.982220
1,1.703101e+10,1.703101e+10,16.300000,282.996230,1.016183,4.055952,0.123175,2.324792,6.503919,2.907989,1.297676,3.666667,1,1.703101e+10,1.703101e+10,0.607989,0.264343,4.205665
2,1.703101e+10,1.703101e+10,17.470588,299.877451,1.095098,4.301471,0.284314,1.947059,6.532843,0.849353,0.860332,1.764706,1,1.703101e+10,1.703101e+10,0.084647,0.036803,1.709685
3,1.703101e+10,1.703101e+10,13.863636,218.310606,0.703788,3.579545,0.590909,2.125000,6.295455,1.168037,0.513771,1.954545,1,1.703101e+10,1.703101e+10,0.031673,0.013771,1.681807
4,1.703101e+10,1.703101e+10,15.692308,326.284799,1.366850,4.922161,0.338828,2.130495,7.391484,1.257564,0.545095,1.923077,1,1.703101e+10,1.703101e+10,0.103718,0.045095,1.802659


In [15]:
# this is a way to convert the chicago data file tract column from a float to a int.. save for later

#test = df_final['Pickup Census Tract'].astype(str)

#int(test[0][:-2])

In [16]:
empty.DESTINATION = empty.DESTINATION.astype(float)
empty.ORIGIN = empty.ORIGIN.astype(float)

In [17]:
rh_final = empty.merge(df_final, how = 'left', left_on = ['ORIGIN', 'DESTINATION','TOD'], right_on = ['Pickup Census Tract', 'Dropoff Census Tract', 'TOD'])

In [18]:
rh_final.head()

,Unnamed: 0,DESTINATION,ORIGIN,TOD,Pickup Census Tract,Dropoff Census Tract,DAY,Trip Seconds,Trip Miles,Fare,...,Additional Charges,Trip Total,PRIVATE_TRIPS,SHARED_TRIPS,Trips Pooled,GEOID_PICKUP,GEOID_DROPOFF,SCALED_SUP_PRIVATE_TRIPS,SCALED_SUP_SHARED_TRIPS,ALL_TRIPS
0,0,1.703184e+10,1.703184e+10,1,1.703184e+10,1.703184e+10,16.000000,387.749862,1.472794,4.937970,...,2.267828,7.290407,9.312559,5.281137,18.387097,1.703184e+10,1.703184e+10,0.119011,0.087588,14.593696
1,1,1.703184e+10,1.703184e+10,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.703184e+10,1.703184e+10,1,1.703184e+10,1.703184e+10,16.200000,1160.900000,12.260000,12.500000,...,1.785000,14.285000,0.629728,0.621234,2.000000,1.703184e+10,1.703184e+10,0.029728,0.021234,1.250962
3,3,1.703184e+10,1.703184e+10,1,1.703184e+10,1.703184e+10,10.500000,1159.000000,11.650000,16.250000,...,2.550000,18.800000,1.003054,0.006872,1.000000,1.703184e+10,1.703184e+10,0.003054,0.006872,1.009926
4,4,1.703184e+10,1.703184e+10,1,1.703184e+10,1.703184e+10,16.666667,1122.333333,10.477778,13.333333,...,1.983333,15.427778,0.784182,0.628931,1.444444,1.703184e+10,1.703184e+10,0.117516,0.184487,1.413114


In [19]:
len(rh_final)

3208005

In [20]:
len(df_final)

660741

In [22]:
rh_final = rh_final.fillna(0)

In [24]:
rh_final.ALL_TRIPS.value_counts()

0.000000    2563533
1.045424          1
1.108933          1
1.791424          1
1.032884          1
             ...   
1.000439          1
1.099716          1
1.004514          1
1.001300          1
1.045207          1
Name: ALL_TRIPS, Length: 644473, dtype: int64

In [25]:
rh_final.to_csv('Outputs/2019_Weekday_Yearly_Ridehail_TOD.csv')

In [28]:
rh_final_day = rh_final[['ORIGIN','DESTINATION','ALL_TRIPS']].groupby(by = ['ORIGIN','DESTINATION'], as_index = False).sum()

0.000000    368736
1.001547         1
1.007739         1
6.343408         1
3.064293         1
             ...  
3.219261         1
2.260745         1
1.004664         1
2.030014         1
1.005966         1
Name: ALL_TRIPS, Length: 272866, dtype: int64

In [10]:
df_raw.to_csv('Inputs/Chicago Ride-Hailing/Raw_Trip_Records_No_Suppressed.csv')

In [11]:
df_all.to_csv('Inputs/Chicago Ride-Hailing/Monthly_Trip_Records_No_Suppressed.csv')

In [40]:
rh_final.TOD.value_counts()

5    641601
4    641601
3    641601
2    641601
1    641601
Name: TOD, dtype: int64

# Data Accuracy Check

In [ ]:
df = pd.read_hdf('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Chicago_TNC_Trips_20.H5', key = 'Weekday_' + str(3))

In [14]:
jan_feb = df[df['MONTH'].isin([1,2])]

In [16]:
miles_over_50 = jan_feb[jan_feb['Trip Miles'] > 50]

In [17]:
len(miles_over_50)

77

In [18]:
miles_over_50.to_csv('Trips_Over_50_Miles.csv')

In [19]:
fare_0 = jan_feb[jan_feb['Fare'] == 0]

In [39]:
fare_0.to_csv('Fare_0_trips.csv')

In [41]:
jan_feb[(jan_feb['Trip Start Timestamp'] == '2019-01-15 08:30:00')&(jan_feb['Pickup Census Tract'] == 17031839700)&(jan_feb['Dropoff Census Tract'] == 17031320100)]

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Fare,Tip,Additional Charges,Trip Total,Shared Trip Authorized,Trips Pooled,YEAR,MONTH,DOW,HOUR
9133667,2019-01-15 08:30:00,2019-01-15 09:00:00,1679.0,5.2,1.703184e+10,1.703132e+10,0.0,0.0,0.67,0.67,True,2.0,2019,1,1,8


In [42]:
miles_over_50.head()

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Fare,Tip,Additional Charges,Trip Total,Shared Trip Authorized,Trips Pooled,YEAR,MONTH,DOW,HOUR
9285742,2019-01-24 08:15:00,2019-01-24 10:30:00,8068.0,58.1,1.703132e+10,1.703184e+10,152.5,0.0,2.55,155.05,False,1.0,2019,1,3,8
9764265,2019-01-01 08:45:00,2019-01-01 10:45:00,7752.0,90.7,1.703108e+10,1.703108e+10,112.5,21.0,5.50,139.00,False,1.0,2019,1,1,8
10872560,2019-02-21 07:45:00,2019-02-21 09:45:00,7645.0,51.7,1.703184e+10,1.703184e+10,80.0,11.0,3.30,94.30,False,1.0,2019,2,3,7
11963121,2019-02-01 08:15:00,2019-02-01 10:00:00,6133.0,53.9,1.703184e+10,1.703128e+10,75.0,0.0,2.55,77.55,False,1.0,2019,2,4,8
12646541,2019-01-25 08:30:00,2019-01-25 10:30:00,7135.0,83.4,1.703108e+10,1.703183e+10,102.5,0.0,2.55,105.05,False,1.0,2019,1,4,8


In [45]:
jan_feb[jan_feb['Trip Total'] == 0]

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Fare,Tip,Additional Charges,Trip Total,Shared Trip Authorized,Trips Pooled,YEAR,MONTH,DOW,HOUR
30217883,2019-01-28 07:30:00,2019-01-28 07:45:00,959.0,2.0,1.703108e+10,1.703107e+10,0.0,0.0,0.0,0.0,True,6.0,2019,1,0,7


In [44]:
len(jan_feb[jan_feb['Fare'] == 0])

32806

In [47]:
trips_5hrs_moore = jan_feb[jan_feb['Trip Seconds'] >= 18000]

In [48]:
trips_5hrs_moore.to_csv('trips_over_5hrs.csv')